In [1]:
import open3d as o3d
import numpy as np
import copy
import utils.alignment as al
import utils.filter as fltr

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Loading and aligning

In [2]:
source = o3d.io.read_point_cloud(r'point_clouds/BaseMug.ply')
#source = fltr.filter_background(source)
target = o3d.io.read_point_cloud(r'point_clouds/BaseMug.ply')

In [3]:
# disturb source
for point in source.points:
    shift = np.random.uniform(low=-100, high=100, size=(3,))
    shift /= np.linalg.norm(shift) * (1/0.0001)
    point += shift

R = source.get_rotation_matrix_from_xyz((np.pi / 2, 0, 0))
source.rotate(R)

PointCloud with 25862 points.

In [4]:
transformation = al.global_registration(source, target)
source_temp = copy.deepcopy(source)
target_temp = copy.deepcopy(target)

transformation = transformation.copy()
# transformation[0,3] = 0.001 # add a tiny bit of offset so we can see alignment otherwise they are perfectly overlapped
print(transformation)
source_temp.transform(transformation)
source_temp.paint_uniform_color([1, 0.706, 0])
target_temp.paint_uniform_color([0, 0.651, 0.929])
o3d.visualization.draw_geometries([source_temp, target_temp])

[Open3D WARNING] Too few correspondences (1983) after mutual filter, fall back to original correspondences.
[[ 9.99978548e-01 -6.26598719e-03  1.90831868e-03 -7.56989817e-05]
 [-1.94075851e-03 -5.17513789e-03  9.99984726e-01 -5.47524462e-02]
 [-6.25601567e-03 -9.99966977e-01 -5.18718764e-03  5.58782780e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


### Comparing

In [5]:
threshold = 0.01

# target_temp.estimate_normals()

mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(target_temp, depth=9)

vertices = o3d.core.Tensor(np.array(mesh.vertices), dtype=o3d.core.float32)
triangles = o3d.core.Tensor(np.array(mesh.triangles), dtype=o3d.core.uint32)

o3d.visualization.draw_geometries([mesh])

scene = o3d.t.geometry.RaycastingScene()
scene.add_triangles(vertices, triangles)

distances = []

for point in source_temp.points:
    query_point = o3d.core.Tensor([point], dtype=o3d.core.float32)
    distance = scene.compute_distance(query_point)
    distances.append(distance.numpy()[0])

distances = np.array(distances)
defect = np.any(distances > threshold)
print(np.max(distances))

print('Defect' if defect else 'No Defect')
print

0.005329449
No Defect


<function print>